In [ ]:
import darts

In [ ]:
import pickle
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from darts.dataprocessing.transformers import Scaler

from darts import TimeSeries, concatenate
from darts.metrics import rmse

torch.manual_seed(3407)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import matplotlib.pyplot as plt

PREFIX = "."

In [ ]:
# pure copy
N_COMPONENTS = 30
BATCH_SIZE = 8

In [ ]:
# Matplotlib customizations.
plt.rc("figure", dpi=300, figsize=(9,3))
plt.rc("font", family="serif", size=6)
plt.rc("legend", edgecolor='none')

In [ ]:
from darts.models import TransformerModel 

In [ ]:
chosen_columns = ['U:0', 'U:1', 'U:2', 'k',]  # check absolute value other U:0, U:1 (done in 01_data_transform.ipynb)
sets = ['train', 'val', 'test']
TRAIN = 499
VAL = 99
TEST = 999 - TRAIN - VAL

In [ ]:
def scale_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))    
    scaled_data = scaler.fit_transform(data)
    pca = PCA(n_components=N_COMPONENTS)
    reduced_data=pca.fit_transform(scaled_data)
    assert reduced_data.shape[1] == N_COMPONENTS    
    timeseries_data = TimeSeries.from_values(data)
    return scaler, pca, reduced_data

In [ ]:
tables = {}
for column in chosen_columns:
    for dataset in sets:
        with open(f'{dataset}_{column}.pkl', 'rb') as f:
            print(f'{dataset}_{column}.pkl')
            if column in tables.keys():
                tables[column] = np.concatenate([tables[column], pickle.load(f)], axis=0)
            else:
                # first time read:
                tables[column] = pickle.load(f)

# discard the first data point for train, details in opinf.ipynb
for i in chosen_columns:    
    tables[i] = tables[i][:,1:]

In [ ]:
with open(f'{PREFIX}/train_p.pkl', 'rb') as f:
    train_data = pickle.load(f)

with open(f'{PREFIX}/val_p.pkl', 'rb') as f:
    val_data = pickle.load(f)

with open(f'{PREFIX}/test_p.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [ ]:
scalers, pca, reduced_data = {}, {}, {}
for column in chosen_columns:
    print(column)
    scalers[column], pca[column], reduced_data[column] = scale_data(tables['U:0'])

In [ ]:
covariates = concatenate([TimeSeries.from_values(reduced_data[i]) for i in chosen_columns if i != 'p'], axis=1)

In [ ]:
covariates

In [ ]:
test_data[:, 1]

In [ ]:
val_data[:, 1]

In [ ]:
train_data[:, 1]

In [ ]:
train_dataset = HEDataset(train_data, train=True)

In [ ]:
val_dataset = HEDataset(val_data, pca=train_dataset.pca, mean=train_dataset.mean, std=train_dataset.std)
test_dataset = HEDataset(test_data, pca=train_dataset.pca, mean=train_dataset.mean, std=train_dataset.std)

In [ ]:
parameters = {
    "num_encoder_layers": [2, 4, 8],
    "num_decoder_layers": [2, 4, 8],
    "dropout": [0.1, 0.3, 0.5],    
    "input_chunk_length": [INPUT_STEP],
    "output_chunk_length": [OUTPUT_STEP],
    "batch_size": [16],
    "n_epochs": [4],
    "d_model": [N_COMPONENTS],
}

big_time_series = TimeSeries.from_values(np.concatenate((train_dataset.normalized_data,
                                                         val_dataset.normalized_data,
                                                         test_dataset.normalized_data),
                                                        axis=0))

In [ ]:
parameters = {
    "num_encoder_layers": [2, 4, 8],
    "num_decoder_layers": [2, 4, 8],
    "dropout": [0.1, 0.3, 0.5],    
    "input_chunk_length": [INPUT_STEP],
    "output_chunk_length": [OUTPUT_STEP],
    "batch_size": [16],
    "n_epochs": [4],
    "d_model": [N_COMPONENTS],
}

best_model, _, _ = TransformerModel.gridsearch(
    parameters = parameters,
    series =big_time_series[:len(train_dataset.normalized_data)],
    val_series = big_time_series[len(train_dataset.normalized_data):len(train_dataset.normalized_data) + len(val_dataset.normalized_data)],
    metric = rmse,
    n_jobs=16,
)
# nhead=N_COMPONENTS,
#     num_encoder_layers=2,
#     num_decoder_layers=2,
#     dim_feedforward=128,
#     dropout=0.1,
#     activation="relu",
#     random_state=3407,
#     save_checkpoints=True,
#     force_reset=True,
#     pl_trainer_kwargs={'enable_progress_bar': True}

In [ ]:
best_model.model_params

In [ ]:
best_model.fit(big_time_series[:len(train_dataset.normalized_data) + len(val_dataset.normalized_data)])

In [ ]:
prediction = best_model.predict(len(big_time_series) - (len(train_dataset.normalized_data) + len(val_dataset.normalized_data)))
predict_original = train_dataset.pca.inverse_transform(prediction.values())
mse(predict_original, test_data, squared=False)

A table for input to choose

| Feature | Test loss          |
|---------|--------------------|
| p       | 110.71740903403541 |
|         |                    |
|         |                    |


In [ ]:
predict_original.shape

In [ ]:
test_data.shape

In [ ]:
def plot_pred(groundtruth, prediction):
    plt.plot(range(groundtruth.shape[0]), groundtruth.mean(1))
    plt.plot(range(groundtruth.shape[0]), prediction.mean(1), alpha=0.5)
    plt.legend(["groundtruth", "prediction"])
plot_pred(test_data, predict_original)